In [15]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Define the URL of the target website
url = 'https://www.allrecipes.com/'

# Define a user-agent to make the request look like it's coming from a web browser
headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:98.0) Gecko/20100101 Firefox/98.0",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1",
        "Sec-Fetch-Dest": "document",
        "Sec-Fetch-Mode": "navigate",
        "Sec-Fetch-Site": "none",
        "Sec-Fetch-User": "?1",
        "Cache-Control": "max-age=0",
    }
# Send an HTTP request to the target URL
response = requests.get(url, headers=headers)

In [16]:
# Check if the request was successful
if response.status_code == 200:
    # Parse the HTML content of the page
    soup = BeautifulSoup(response.content, 'html.parser')

    #print(soup)
    
    # Extract the job listings
    recipes_cards = soup.find_all('a', class_='comp mntl-card-list-items mntl-document-card mntl-card card--image-top card card--no-image')
    recipes_urls = [card['href'] for card in recipes_cards]

    recipes_data = []
    
    # Print the extracted data
    for url in recipes_urls:
        recipe_page = requests.get(url, headers=headers)
        recipe_soup = BeautifulSoup(recipe_page.content, 'html.parser')
        
        recipe_title = recipe_soup.find('h1', class_='article-heading type--lion')
        
        recipe_ingredients_list = recipe_soup.find('ul', class_ = 'mm-recipes-structured-ingredients__list')
        recipe_ingredients_elmnts = recipe_ingredients_list.find_all('p')
        recipe_ingredients = [[p.text] for p in recipe_ingredients_elmnts]
        
        recipes_steps_list = recipe_soup.find('ol', class_ = 'comp mntl-sc-block mntl-sc-block-startgroup mntl-sc-block-group--OL')
        recipes_steps_elmnt = recipes_steps_list.find_all('p')
        recipes_steps = [[p.text] for p in recipes_steps_elmnt]
        
        recipes_data.append({
            'Title': recipe_title,
            'Ingredientes': recipe_ingredients,
            'Steps': recipes_steps
        })

    df = pd.DataFrame(recipes_data)
    print(df)

else:
    print(f'Failed to retrieve the webpage. Status code: {response.status_code}')

                                        Title  \
0                      [Chicken Noodle Salad]   
1                       [Asian Lettuce Wraps]   
2                       [Ranch Chicken Tacos]   
3                         [Salmon Cobb Salad]   
4                [Summer Watermelon Gazpacho]   
5               [Summer Grilled Shrimp Salad]   
6        [Chef John's Classic Macaroni Salad]   
7                               [Brown Bears]   
8                      [Seven-Layer Taco Dip]   
9            [Homemade Pickled Ginger (Gari)]   
10     [Big Ray's Cherry Rhubarb Coffee Bars]   
11  [Firecracker Shrimp Roll with Crab Aioli]   

                                         Ingredientes  \
0   [[3 cloves garlic, crushed], [½ cup seasoned r...   
1   [[16  Boston Bibb or butter lettuce leaves], [...   
2   [[½ cup ranch dressing], [¼ cup reduced-fat so...   
3   [[¾ cup buttermilk], [½ cup mayonnaise], [¼ cu...   
4   [[4 cups cubed seeded watermelon], [2  roma (p...   
5   [[1 tablespoon o